In [61]:
import pandas as pd
import numpy as np
from utils.Data import Data
from utils.Camera import Camera
from utils.Plot import Plot

import plotly.graph_objects as go

In [62]:
data = Data()
camera = Camera('data/camera.dat')

INFO - Loading trajectory data from data//trajectory.dat
INFO - 0.00 [s] - Trajectory data loaded successfully!
INFO - Loading world data from data//world.dat
INFO - 0.04 [s] - World data loaded successfully!
INFO - Loading measurements data from data//meas-*.dat
INFO - 0.04 [s] - Measurements data loaded successfully!
INFO - Loading camera parameters from data/camera.dat
INFO - 0.00s - Camera parameters loaded successfully.


In [89]:
def data_association(measurement_1, measurement_2):
    gt_pose_1 = measurement_1['gt_pose']
    odom_pose_1 = measurement_1['odom_pose']
    points_1 = measurement_1['points']

    gt_pose_2 = measurement_2['gt_pose']
    odom_pose_2 = measurement_2['odom_pose']
    points_2 = measurement_2['points']

    matches = {'actual_point_id_1':[], 'appearance_1':[], 'image_point_1':[], 'actual_point_id_2':[], 'appearance_2':[], 'image_point_2':[]}

    for i in range(len(points_1)):
        point_1 = points_1[i]
        actual_point_1 = point_1['actual_point_id']
        image_point_1 = point_1['image_point']
        appearance_1 = point_1['appearance']

        for j in range(len(points_2)):
            point_2 = points_2[j]
            actual_point_2 = point_2['actual_point_id']
            image_point_2 = point_2['image_point']
            appearance_2 = point_2['appearance']

            if appearance_1 == appearance_2:
                matches['actual_point_id_1'].append(actual_point_1)
                matches['appearance_1'].append(appearance_1)
                matches['image_point_1'].append(image_point_1)
                matches['actual_point_id_2'].append(actual_point_2)
                matches['appearance_2'].append(appearance_2)
                matches['image_point_2'].append(image_point_2)

    return gt_pose_1, odom_pose_1, gt_pose_2, odom_pose_2, matches

In [91]:
meas_0 = data.get_measurements(0)
meas_1 = data.get_measurements(1)

gt_pose_1, odom_pose_1, gt_pose_2, odom_pose_2, matches = data_association(meas_0, meas_1)
print('gt_pose_1', gt_pose_1)
print('odom_pose_1', odom_pose_1)
print('gt_pose_2', gt_pose_2)
print('odom_pose_2', odom_pose_2)
matches_df = pd.DataFrame(matches)
matches_df

gt_pose_1 [0.0, 0.0, 0.0]
odom_pose_1 [0.00160159, 0.0, -0.000259093]
gt_pose_2 [0.200426, 0.0, 0.0]
odom_pose_2 [0.200147, 0.00177571, 0.018434]


,actual_point_id_1,appearance_1,image_point_1,actual_point_id_2,appearance_2,image_point_2
0,6,"[-0.668052, -0.119791, 0.76015, 0.658402, -0.3...","[522.119, 187.968]",6,"[-0.668052, -0.119791, 0.76015, 0.658402, -0.3...","[539.005, 183.622]"
1,14,"[-0.80072, 0.0616159, 0.514588, -0.39141, 0.98...","[442.949, 142.838]",14,"[-0.80072, 0.0616159, 0.514588, -0.39141, 0.98...","[463.689, 126.449]"
2,15,"[0.746543, 0.662075, 0.958868, 0.487622, 0.806...","[67.7619, 175.604]",15,"[0.746543, 0.662075, 0.958868, 0.487622, 0.806...","[56.6646, 172.771]"
3,17,"[-0.439916, 0.0922137, 0.438537, -0.773439, -0...","[306.371, 119.738]",17,"[-0.439916, 0.0922137, 0.438537, -0.773439, -0...","[305.329, 110.546]"
4,35,"[0.104633, 0.839182, 0.371973, 0.619571, 0.395...","[509.615, 216.308]",35,"[0.104633, 0.839182, 0.371973, 0.619571, 0.395...","[518.356, 215.216]"
...,...,...,...,...,...,...
110,939,"[-0.426205, -0.741321, 0.567446, 0.887447, 0.2...","[265.905, 212.571]",939,"[-0.426205, -0.741321, 0.567446, 0.887447, 0.2...","[263.409, 211.305]"
111,963,"[-0.0250311, -0.80992, -0.385503, 0.413624, -0...","[302.141, 194.465]",963,"[-0.0250311, -0.80992, -0.385503, 0.413624, -0...","[300.962, 191.459]"
112,971,"[0.683682, -0.00590533, 0.276129, -0.692478, 0...","[340.454, 198.249]",971,"[0.683682, -0.00590533, 0.276129, -0.692478, 0...","[341.861, 195.379]"
113,979,"[0.0400809, 0.470501, -0.1505, 0.457585, 0.206...","[110.052, 55.203]",979,"[0.0400809, 0.470501, -0.1505, 0.457585, 0.206...","[81.2067, 29.8137]"


In [65]:
def compute_R_t(set1, set2):

    '''
    Given two sets of points, set1 and set2, which are the corresponding points in two different coordinate systems,
    this function returns the rotation and translation matrix that best aligns the two sets of points.
    '''

    # Compute the centroid of each set of points
    x_0 = np.mean(set1, axis=0)
    y_0 = np.mean(set2, axis=0)

    # Subtract the centroid from each set of points
    set1 = set1 - x_0
    set2 = set2 - y_0

    # Calculate the covariance matrix of the two sets of points
    H = np.dot(set1.T, set2)
    
    # Use the SVD to calculate the rotation matrix
    U, S, Vt = np.linalg.svd(H)

    # Calculate the rotation matrix
    # R = np.dot(Vt.T, U.T)
    R = np.dot(U, Vt)

    # Calculate the translation matrix
    # t = y_0 - np.dot(R, x_0)
    t = y_0 - np.dot(x_0, R)

    return R, t

In [66]:
meas = data.get_measurements()[0]
gt_pose = meas['gt_pose']
point = meas['points'][0]
image_point = point['image_point']
appearance = point['appearance']
id = point['actual_point_id']

print('Ground truth robot pose:', gt_pose)
print('id:', id)
print('Image point:', image_point)
print('Appearance:', appearance)
print('')

world_data = data.get_world(6)
ground_truth_point = world_data['landmark_position']
ground_truth_apperance = world_data['landmark_appearance']

print('Ground truth id:', 6)
print('Ground truth point:', ground_truth_point)
print('Ground truth appearance:', ground_truth_apperance)
print('')

estimated_image_point = camera.world_to_pixel_projection(ground_truth_point)
print('Estimated image point:', estimated_image_point)

Ground truth robot pose: [0.0, 0.0, 0.0]
id: 6
Image point: [522.119, 187.968]
Appearance: [-0.668052, -0.119791, 0.76015, 0.658402, -0.339326, -0.542064, 0.786745, -0.29928, 0.37334, 0.912936]

Ground truth id: 6
Ground truth point: [2.79958, -2.91903, 0.751446]
Ground truth appearance: [-0.668052, -0.119791, 0.76015, 0.658402, -0.339326, -0.542064, 0.786745, -0.29928, 0.37334, 0.912936]

Estimated image point: [522.1193423553035, 187.96841028166088]


In [108]:
meas_0 = data.get_measurements()[0]
meas_1 = data.get_measurements()[1]

gt_pose_0 = meas_0['gt_pose']
point_0 = meas_0['points'][0]
image_point_0 = point_0['image_point']
appearance_0 = point_0['appearance']
id_0 = point_0['actual_point_id']

gt_pose_1 = meas_1['gt_pose']
point_1 = meas_1['points'][0]
image_point_1 = point_1['image_point']
appearance_1 = point_1['appearance']
id_1 = point_1['actual_point_id']


print('Ground truth robot pose 0:', gt_pose_0)
print('id 0:', id_0)
print('Image point 0:', image_point_0)
print('Appearance 0:', appearance_0)
print('')
print('Ground truth robot pose 1:', gt_pose_1)
print('id 1:', id_1)
print('Image point 1:', image_point_1)
print('Appearance 1:', appearance_1)
print('')

world_data = data.get_world(6)
ground_truth_apperance = world_data['landmark_appearance']
ground_truth_point = np.array(world_data['landmark_position'])
ground_truth_point_0 = ground_truth_point - np.array(gt_pose_0)
ground_truth_point_1 = ground_truth_point - np.array(gt_pose_1)

print('Ground truth id:', 6)
print('Ground truth point:', ground_truth_point)
print('Ground truth point 0:', ground_truth_point_0)
print('Ground truth point 1:', ground_truth_point_1)
print('Ground truth appearance:', ground_truth_apperance)
print('')

estimated_image_point_0 = camera.world_to_pixel_projection(ground_truth_point)
estimated_image_point_1 = camera.world_to_pixel_projection(ground_truth_point_1)

print('Estimated image point 0:', estimated_image_point_0)
print('Estimated image point 1:', estimated_image_point_1)
print('')

###############
R = camera.get_camera_transform()[:3, :3]
t = camera.get_camera_transform()[:3, 3]

image_point_0_hom = np.array([image_point_0[0], image_point_0[1], 1])
pc_0 = np.dot(np.linalg.inv(camera.get_camera_matrix()), image_point_0_hom)
pw_0 = t + np.dot(R, pc_0)
cam_0 = np.array(gt_pose_0).T
cam_world_0 = t + np.dot(R, cam_0)
vector_0 = (pw_0 - cam_world_0) / np.linalg.norm(pw_0 - cam_world_0)
estimated_world_point_0 = cam_world_0 + 3.98*vector_0

image_point_1_hom = np.array([image_point_1[0], image_point_1[1], 1])
pc_1 = np.dot(np.linalg.inv(camera.get_camera_matrix()), image_point_1_hom)
pw_1 = t + np.dot(R, pc_1)
cam_1 = np.array(gt_pose_1).T
cam_world_1 = t + np.dot(R, cam_1)
vector_1 = (pw_1 - cam_world_1) / np.linalg.norm(pw_1 - cam_world_1)
estimated_world_point_1 = cam_world_1 + 3.98*vector_1

print('Estimated world point 0:', estimated_world_point_0)
print('Estimated world point 1:', estimated_world_point_1)

# Plot
fig = go.Figure()

# Add ground truth point
fig.add_trace(go.Scatter3d(x=[ground_truth_point[0]], y=[ground_truth_point[1]], z=[ground_truth_point[2]], mode='markers', name='Ground truth point', marker=dict(size=5, color='blue')))
fig.add_trace(go.Scatter3d(x=[estimated_world_point_0[0]], y=[estimated_world_point_0[1]], z=[estimated_world_point_0[2]], mode='markers', name='Estimated world point 0', marker=dict(size=5, color='green')))
fig.add_trace(go.Scatter3d(x=[estimated_world_point_1[0]], y=[estimated_world_point_1[1]], z=[estimated_world_point_1[2]], mode='markers', name='Estimated world point 1', marker=dict(size=5, color='red')))
# add cam_0 
fig.add_trace(go.Scatter3d(x=[cam_world_0[0]], y=[cam_world_0[1]], z=[cam_world_0[2]], mode='markers', name='cam_world_0', marker=dict(size=5, color='yellow')))
# add cam_1
fig.add_trace(go.Scatter3d(x=[cam_world_1[0]], y=[cam_world_1[1]], z=[cam_world_1[2]], mode='markers', name='cam_world_1', marker=dict(size=5, color='orange')))

fig.update_layout(scene=dict(aspectmode="cube"))    
fig.show()

Ground truth robot pose 0: [0.0, 0.0, 0.0]
id 0: 6
Image point 0: [522.119, 187.968]
Appearance 0: [-0.668052, -0.119791, 0.76015, 0.658402, -0.339326, -0.542064, 0.786745, -0.29928, 0.37334, 0.912936]

Ground truth robot pose 1: [0.200426, 0.0, 0.0]
id 1: 6
Image point 1: [539.005, 183.622]
Appearance 1: [-0.668052, -0.119791, 0.76015, 0.658402, -0.339326, -0.542064, 0.786745, -0.29928, 0.37334, 0.912936]

Ground truth id: 6
Ground truth point: [ 2.79958  -2.91903   0.751446]
Ground truth point 0: [ 2.79958  -2.91903   0.751446]
Ground truth point 1: [ 2.599154 -2.91903   0.751446]
Ground truth appearance: [-0.668052, -0.119791, 0.76015, 0.658402, -0.339326, -0.542064, 0.786745, -0.29928, 0.37334, 0.912936]

Estimated image point 0: [522.1193423553035, 187.96841028166088]
Estimated image point 1: [539.0044490682966, 183.62167664101597]

Estimated world point 0: [ 2.79934999 -2.91876678  0.75138544]
Estimated world point 1: [ 2.92647335 -2.97125483  0.85396175]


In [16]:
data.print_trajectory(0)

Trajectory data: 
pose_id: 0
  ground_truth_pose: [0.0, 0.0, 0.0]
  odometry_pose:     [0.00160159, 0.0, -0.000259093]


In [17]:
data.print_world(6)

World data: 
landmark_id: 6
  landmark_position:   [2.79958, -2.91903, 0.751446]
  landmark_appearance: [-0.668052, -0.119791, 0.76015, 0.658402, -0.339326, -0.542064, 0.786745, -0.29928, 0.37334, 0.912936]


In [7]:
data.print_measurements(0)

Measurements data: 
sequence_id: 0
  ground_truth_pose: [0.0, 0.0, 0.0]
  odometry_pose:     [0.00160159, 0.0, -0.000259093]
  Points: 
    point_id: 0
      actual_point_id: 6
      image_point:     [522.119, 187.968]
      appearance:      [-0.668052, -0.119791, 0.76015, 0.658402, -0.339326, -0.542064, 0.786745, -0.29928, 0.37334, 0.912936]
    point_id: 1
      actual_point_id: 14
      image_point:     [442.949, 142.838]
      appearance:      [-0.80072, 0.0616159, 0.514588, -0.39141, 0.984457, 0.153942, 0.755228, 0.495619, 0.25782, -0.929158]
    point_id: 2
      actual_point_id: 15
      image_point:     [67.7619, 175.604]
      appearance:      [0.746543, 0.662075, 0.958868, 0.487622, 0.806733, 0.967191, 0.333761, -0.00548297, -0.672064, 0.660024]
    point_id: 3
      actual_point_id: 17
      image_point:     [306.371, 119.738]
      appearance:      [-0.439916, 0.0922137, 0.438537, -0.773439, -0.0570331, 0.18508, 0.888636, -0.0981649, -0.327298, 0.695369]
    point_id: 4
   

In [19]:
gt1, odom1, points1 = compute_points(data.get_measurements(0))
points1_df = pd.DataFrame(points1)
points1_df

,point_id,image_point,camera_point,world_point,appearance
0,6,"[522.119, 187.968]","[-1.7742687673611113, -1.3321288888888891, 1.0]","[1.2, 1.7742687673611113, 1.3321288888888891]","[-0.668052, -0.119791, 0.76015, 0.658402, -0.3..."
1,14,"[442.949, 142.838]","[-1.7756432465277778, -1.3310842129629632, 1.0]","[1.2, 1.7756432465277778, 1.3310842129629632]","[-0.80072, 0.0616159, 0.514588, -0.39141, 0.98..."
2,15,"[67.7619, 175.604]","[-1.7821569114583335, -1.3318426851851852, 1.0]","[1.2, 1.7821569114583335, 1.3318426851851852]","[0.746543, 0.662075, 0.958868, 0.487622, 0.806..."
3,17,"[306.371, 119.738]","[-1.7780143923611111, -1.330549490740741, 1.0]","[1.2, 1.7780143923611111, 1.330549490740741]","[-0.439916, 0.0922137, 0.438537, -0.773439, -0..."
4,35,"[509.615, 216.308]","[-1.7744858506944445, -1.3327849074074076, 1.0]","[1.2, 1.7744858506944445, 1.3327849074074076]","[0.104633, 0.839182, 0.371973, 0.619571, 0.395..."
...,...,...,...,...,...
122,942,"[563.308, 146.283]","[-1.7735536805555556, -1.3311639583333335, 1.0]","[1.2, 1.7735536805555556, 1.3311639583333335]","[-0.770661, -0.313814, 0.627311, -0.489216, 0...."
123,963,"[302.141, 194.465]","[-1.7780878298611111, -1.3322792824074075, 1.0]","[1.2, 1.7780878298611111, 1.3322792824074075]","[-0.0250311, -0.80992, -0.385503, 0.413624, -0..."
124,971,"[340.454, 198.249]","[-1.7774226736111112, -1.3323668750000002, 1.0]","[1.2, 1.7774226736111112, 1.3323668750000002]","[0.683682, -0.00590533, 0.276129, -0.692478, 0..."
125,979,"[110.052, 55.203]","[-1.7814227083333334, -1.329055625, 1.0]","[1.2, 1.7814227083333334, 1.329055625]","[0.0400809, 0.470501, -0.1505, 0.457585, 0.206..."


In [113]:
gt2, odom2, points2 = compute_points(data.get_measurements(1))
points2_df = pd.DataFrame(points2)
points2_df

,point_id,image_point,camera_point,world_point,appearance
0,6,"[539.005, 183.622]","[-1.773975607638889, -1.3320282870370372, 1.0]","[1.2, 1.773975607638889, 1.3320282870370372]","[-0.668052, -0.119791, 0.76015, 0.658402, -0.3..."
1,14,"[463.689, 126.449]","[-1.7752831770833335, -1.3307048379629631, 1.0]","[1.2, 1.7752831770833335, 1.3307048379629631]","[-0.80072, 0.0616159, 0.514588, -0.39141, 0.98..."
2,15,"[56.6646, 172.771]","[-1.7823495729166667, -1.3317771064814816, 1.0]","[1.2, 1.7823495729166667, 1.3317771064814816]","[0.746543, 0.662075, 0.958868, 0.487622, 0.806..."
3,17,"[305.329, 110.546]","[-1.778032482638889, -1.330336712962963, 1.0]","[1.2, 1.778032482638889, 1.330336712962963]","[-0.439916, 0.0922137, 0.438537, -0.773439, -0..."
4,35,"[518.356, 215.216]","[-1.7743340972222224, -1.3327596296296298, 1.0]","[1.2, 1.7743340972222224, 1.3327596296296298]","[0.104633, 0.839182, 0.371973, 0.619571, 0.395..."
...,...,...,...,...,...
119,939,"[263.409, 211.305]","[-1.7787602604166668, -1.3326690972222224, 1.0]","[1.2, 1.7787602604166668, 1.3326690972222224]","[-0.426205, -0.741321, 0.567446, 0.887447, 0.2..."
120,963,"[300.962, 191.459]","[-1.7781082986111112, -1.3322096990740742, 1.0]","[1.2, 1.7781082986111112, 1.3322096990740742]","[-0.0250311, -0.80992, -0.385503, 0.413624, -0..."
121,971,"[341.861, 195.379]","[-1.777398246527778, -1.332300439814815, 1.0]","[1.2, 1.777398246527778, 1.332300439814815]","[0.683682, -0.00590533, 0.276129, -0.692478, 0..."
122,979,"[81.2067, 29.8137]","[-1.781923494791667, -1.3284679097222223, 1.0]","[1.2, 1.781923494791667, 1.3284679097222223]","[0.0400809, 0.470501, -0.1505, 0.457585, 0.206..."


In [109]:
p = camera.camera_to_world(points2['camera_point'][0])
p

array([1.2       , 1.77397561, 1.33202829])

In [21]:
matches = data_association(points1, points2)
matches_df = pd.DataFrame(matches).T
matches_df.columns = ['world_point_1', 'world_point_2']
matches_df

,world_point_1,world_point_2
6,"[1.2, 1.7742687673611113, 1.3321288888888891]","[1.2, 1.773975607638889, 1.3320282870370372]"
14,"[1.2, 1.7756432465277778, 1.3310842129629632]","[1.2, 1.7752831770833335, 1.3307048379629631]"
15,"[1.2, 1.7821569114583335, 1.3318426851851852]","[1.2, 1.7823495729166667, 1.3317771064814816]"
17,"[1.2, 1.7780143923611111, 1.330549490740741]","[1.2, 1.778032482638889, 1.330336712962963]"
35,"[1.2, 1.7744858506944445, 1.3327849074074076]","[1.2, 1.7743340972222224, 1.3327596296296298]"
...,...,...
939,"[1.2, 1.7787169270833334, 1.332698402777778]","[1.2, 1.7787602604166668, 1.3326690972222224]"
963,"[1.2, 1.7780878298611111, 1.3322792824074075]","[1.2, 1.7781082986111112, 1.3322096990740742]"
971,"[1.2, 1.7774226736111112, 1.3323668750000002]","[1.2, 1.777398246527778, 1.332300439814815]"
979,"[1.2, 1.7814227083333334, 1.329055625]","[1.2, 1.781923494791667, 1.3284679097222223]"


In [87]:
world_data = data.get_world()
world_data_df = pd.DataFrame(world_data).T
world_data_df

,landmark_position,landmark_appearance
0,"[6.80375, -2.11234, 1.1324]","[0.59688, 0.823295, -0.604897, -0.329554, 0.53..."
1,"[0.268018, 9.04459, 1.66478]","[0.271423, 0.434594, -0.716795, 0.213938, -0.9..."
2,"[-7.40419, -7.82382, 1.9957]","[-0.563486, 0.0258648, 0.678224, 0.22528, -0.4..."
3,"[5.42715, 0.534899, 1.07966]","[-0.199543, 0.783059, -0.433371, -0.295083, 0...."
4,"[-6.15572, 3.26454, 1.56093]","[-0.302214, -0.871657, -0.959954, -0.0845965, ..."
...,...,...
995,"[9.02387, 8.75512, 0.978687]","[-0.949754, 0.805714, 0.564985, 0.752124, 0.99..."
996,"[8.69677, -1.81871, 1.52648]","[0.24774, 0.514616, 0.460844, 0.943842, 0.6594..."
997,"[1.38236, 3.42996, 0.112917]","[0.957896, -0.947002, -0.0411545, 0.833408, -0..."
998,"[-5.10542, -5.3237, 1.73017]","[-0.417916, 0.0843985, -0.995409, 0.400214, -0..."


In [103]:
set1 = []
world_points_set1 = []

for key, value in points1_df.iterrows():
    world_point = value['world_point']
    appearance = value['appearance']

    for key1, value1 in world_data_df.iterrows():
        if value1['landmark_appearance'] == appearance:
            set1.append(list(world_point))
            world_points_set1.append(value1['landmark_position'])

In [104]:
set1

[[1.2, 1.7742687673611113, 1.3321288888888891],
 [1.2, 1.7756432465277778, 1.3310842129629632],
 [1.2, 1.7821569114583335, 1.3318426851851852],
 [1.2, 1.7780143923611111, 1.330549490740741],
 [1.2, 1.7744858506944445, 1.3327849074074076],
 [1.2, 1.7818896024305557, 1.3331910648148149],
 [1.2, 1.7737049479166669, 1.3310944444444446],
 [1.2, 1.7780748784722225, 1.3318814814814817],
 [1.2, 1.7801025347222224, 1.3315655555555557],
 [1.2, 1.7818908836805556, 1.3323869444444445],
 [1.2, 1.7751735937500002, 1.3322701157407408],
 [1.2, 1.7738641666666668, 1.3324375925925926],
 [1.2, 1.776467482638889, 1.3319089583333334],
 [1.2, 1.7756272395833335, 1.332818587962963],
 [1.2, 1.7812847222222223, 1.331671875],
 [1.2, 1.7783478993055557, 1.332345625],
 [1.2, 1.7733953125000002, 1.3327167592592595],
 [1.2, 1.7829816024305556, 1.3330780787037038],
 [1.2, 1.7755722916666667, 1.3318697222222224],
 [1.2, 1.7820115190972223, 1.3331538425925928],
 [1.2, 1.782399703125, 1.3332103240740742],
 [1.2, 1.7793

In [107]:
import plotly.graph_objects as go
# plot the points of the set 1
fig = go.Figure()

fig.add_trace(go.Scatter3d(x=[x[0] for x in set1], y=[x[1] for x in set1], z=[x[2] for x in set1], mode='markers', marker=dict(size=5, color='blue', opacity=0.5)))
fig.add_trace(go.Scatter3d(x=[x[0] for x in world_points_set1], y=[x[1] for x in world_points_set1], z=[x[2] for x in world_points_set1], mode='markers', marker=dict(size=5, color='red', opacity=0.5)))

fig.show()

In [58]:
t_0 = np.zeros(3)
R_0 = np.eye(3)

vo_poses = []
gt_poses = []
odom_poses = []

for i in range(120):
    measurement_0 = data.get_measurements(i)
    measurement_1 = data.get_measurements(i+1)

    gt_pose_0, odom_pose_0, points_0 = compute_points(measurement_0)
    gt_pose_1, odom_pose_1, points_1 = compute_points(measurement_1)

    matches = data_association(points_0, points_1)
    set1 = np.array([matches[key][0] for key in matches])
    set2 = np.array([matches[key][1] for key in matches])

    R_0_1, t_0_1 = compute_R_t(set1, set2)
    
    vo_pose_1 = np.dot(R_0_1, t_0) + t_0_1

    rel_gt = np.array(gt_pose_1) - np.array(gt_pose_0)
    rel_vo = vo_pose_1 - t_0

    vo_poses.append(vo_pose_1)
    gt_poses.append(gt_pose_1)
    odom_poses.append(odom_pose_1)

    t_0 = vo_pose_1

In [59]:
print(f'Ground Truth Trajectory: \n {gt_poses}')
print(f'Odometry Trajectory: \n {odom_poses}')
print(f'Visual Odometry Trajectory: \n {list(vo_poses)}')

Ground Truth Trajectory: 
 [[0.200426, 0.0, 0.0], [0.40085, 0.0, 0.0], [0.601274, 0.0, 0.0], [0.801698, 0.0, 0.0], [1.00244, 0.0, 0.0], [1.20254, 0.0, 0.0], [1.40263, 0.0, 0.0], [1.60272, 0.0, 0.0], [1.80282, 0.0, 0.0], [2.00291, 0.0, 0.0], [2.20353, 0.0, 0.0], [2.40355, 0.0, 0.0], [2.60358, 0.0, 0.0], [2.8036, 0.0, 0.0], [3.00362, 0.0, 0.0], [3.20364, 0.0, 0.0], [3.40319, 0.0151316, 0.188353], [3.56783, 0.0638916, 0.388423], [3.71949, 0.144398, 0.58849], [3.85213, 0.253439, 0.788559], [3.96045, 0.386664, 0.988628], [4.04013, 0.538759, 1.1887], [4.08799, 0.703654, 1.38877], [4.1131, 0.902461, 1.45456], [4.13633, 1.10141, 1.45456], [4.15956, 1.30036, 1.45456], [4.18278, 1.4993, 1.45456], [4.20601, 1.69824, 1.45456], [4.22923, 1.89719, 1.45456], [4.25246, 2.09613, 1.45456], [4.27568, 2.29488, 1.45456], [4.29899, 2.49425, 1.45456], [4.32229, 2.69361, 1.45456], [4.34559, 2.89298, 1.45456], [4.36889, 3.09234, 1.45456], [4.39219, 3.29171, 1.45456], [4.41549, 3.49107, 1.45456], [4.4388, 3.690

In [60]:
max_points = 120
vo_trajectory_to_plot = vo_poses[:max_points]
gt_trajectory_to_plot = gt_poses[:max_points]
odom_trajectory_to_plot = odom_poses[:max_points]

plot = Plot(title='Trajectories: Ground Truth, Odometry and Estimated', cut_axis=False)
plot.add_trajectory(gt_trajectory_to_plot, 'Ground Truth', 'green', 2)
plot.add_trajectory(odom_trajectory_to_plot, 'Odometry', 'blue', 2)
plot.add_trajectory(vo_trajectory_to_plot, 'Estimated', 'red', 2)
plot.show()

In [61]:
max_points = 120
vo_trajectory_to_plot = vo_poses[:max_points]
gt_trajectory_to_plot = gt_poses[:max_points]
odom_trajectory_to_plot = odom_poses[:max_points]

plot = Plot(title='Trajectories: Ground Truth, Odometry and Estimated', cut_axis=False)
plot.add_trajectory(vo_trajectory_to_plot, 'Estimated', 'red', 2)
plot.show()